# [IAPR][iapr]: Final project - Chocolate Recognition


**Moodle group ID:** 50  
**Kaggle challenge:** Classic  
**Kaggle team name (exact):** "ChocoBlabla"  

**Author 1 (SCIPER):** *Louis Cuendet (xxxxxx)*  
**Author 2 (SCIPER):** *Timur Ünver (xxxxx)*  
**Author 3 (SCIPER):** *Adrien Boschung (327221)*  

**Due date:** 21.05.2025 (11:59 pm)


## Key Submission Guidelines:
- **Before submitting your notebook, <span style="color:red;">rerun</span> it from scratch!** Go to: `Kernel` > `Restart & Run All`
- **Only groups of three will be accepted**, except in exceptional circumstances.


[iapr]: https://github.com/LTS5/iapr2025

---

## Segmentation

## Classification

In this section, we will detail which chocolate descriptors we decide to keep or not for classification and the reasons why.

Our first idea was to first clusterize the 